In [1]:

# standard library imports
import git, os, sys

git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

# local imports
from fight_stat_helpers import *
from data_handler import DataHandler

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


# NEW FEATURES

- Dominance Score for a single fight: (total score of fighter based on events in the fight) - (total score of opponent based on events in the fight)
- Average Dominance over time scales
- Fighter Scores
- Fight Math
- Fighter Score increments based on closeness of fights with opponents with high scores 

# Other things to try
- with or without regularization
- with or without scaling
- with or without sum features (seems like only age needs sum features)
- with or without squared features (seems like only age needs squared features)
- Decide whether to include SDEC in bullshit or not
- Do linear reg on draws and SDEC and see how this compares to log reg plane (hypothesis: they are the same plane since close fights should be near the plane boundary)

In [2]:
dh = DataHandler()
# get reported derived doubled data
ufc_fights_reported_derived_doubled = dh.get('ufc_fights_reported_derived_doubled')

In [3]:
ufc_fights_predictive_flattened_diffs = dh.make_ufc_fights_predictive_flattened_diffs(ufc_fights_reported_derived_doubled)
ufc_fights_predictive_flattened_diffs = dh.clean_ufc_fights_for_winner_prediction(ufc_fights_predictive_flattened_diffs)

In [4]:
# show division counts
print(ufc_fights_predictive_flattened_diffs['division'].value_counts())

division
Lightweight              1059
Welterweight             1015
Middleweight              819
Featherweight             668
Bantamweight              590
Light Heavyweight         576
Heavyweight               524
Flyweight                 304
Women's Strawweight       273
Women's Flyweight         222
Women's Bantamweight      178
Catch Weight               62
Women's Featherweight      26
Name: count, dtype: int64


In [5]:
# filter by division. start with just lightweight and featherweight
division_mask = ufc_fights_predictive_flattened_diffs['division'].isin(['Bantamweight','Flyweight','Lightweight', 'Featherweight'])
ufc_fights_predictive_flattened_diffs_light_feather = ufc_fights_predictive_flattened_diffs[division_mask]
ufc_fights_predictive_flattened_diffs_light_feather.shape

(2621, 325)

In [6]:
# drop the 'fighter_result' column and opponent_result column as it is not needed for the model and also fighter and opponent columns
X = ufc_fights_predictive_flattened_diffs_light_feather.drop(columns=['fighter', 'opponent', 'method', 'division'])
# KEEP result in X as this is what model_score is expecting for better or for worse
y = X['result']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91)

# Use the entire data set with all features as a baseline

In [8]:
all_features = X_train.columns.tolist()
# remove result from all_features
if 'result' in all_features:
    all_features.remove('result')
model_test_score(X_train, X_test, all_features, _max_iter = 20000, scaled=True)

Training set size: (2096, 321) accuracy: 0.6970419847328244
Test set size: (525, 321) accuracy: 0.6361904761904762
Test set neg log loss: -0.6533803254485624. Probability to observe data given model: 0.5202840713828026


In [9]:
best_features = additive_greedy(X_train, X_test, search_doubles=True, _max_iter=30000)

Starting with an empty feature set.
Added single feature: age_diff
Negative log loss on training set: -0.6680667206713895
Training set size: (2096, 321) accuracy: 0.5958969465648855
Test set size: (525, 321) accuracy: 0.5714285714285714
Test set neg log loss: -0.6831875397889374. Probability to observe data given model: 0.5050047015303945
Added single feature: l5y_wins_diff
Negative log loss on training set: -0.6522843641261508
Training set size: (2096, 321) accuracy: 0.6054389312977099
Test set size: (525, 321) accuracy: 0.6019047619047619
Test set neg log loss: -0.6702062090531125. Probability to observe data given model: 0.5116030697239803
Added single feature: l5y_wins_wins_diff
Negative log loss on training set: -0.6481502443171775
Training set size: (2096, 321) accuracy: 0.6211832061068703
Test set size: (525, 321) accuracy: 0.6209523809523809
Test set neg log loss: -0.6617221909981301. Probability to observe data given model: 0.5159619838117472
Added single feature: l5y_overall_

In [10]:
model_test_score(X_train, X_test, best_features, _max_iter = 30000, scaled=True)

Training set size: (2096, 321) accuracy: 0.6770038167938931
Test set size: (525, 321) accuracy: 0.6552380952380953
Test set neg log loss: -0.6426414482560229. Probability to observe data given model: 0.5259014462957847
